In [1]:
(version)

"7.7"

In [2]:
#lang iracket/lang #:require bhdl/splicing #:reader bhdl/splicing

In [3]:
(list (list 1 2 3) ..)

'(1 2 3)

In [4]:
(putenv "BHDL_LIBRARY_PATH" "~/bhdl/bhdl-footprints")
(getenv "BHDL_LIBRARY_PATH")

"~/bhdl/bhdl-footprints"

In [5]:
(require racket/list
         racket/format
         (prefix-in pict: pict))

In [6]:
(require bhdl)

DEBUG:  unrecognized shape:  SVGNODE
DEBUG:  unrecognized shape:  SVGNODE
DEBUG:  unrecognized shape:  SVGNODE
DEBUG:  unrecognized shape:  ARC
DEBUG:  unrecognized shape:  ARC
DEBUG:  unrecognized shape:  SVGNODE
DEBUG:  unrecognized shape:  SVGNODE
DEBUG:  unrecognized shape:  SVGNODE
DEBUG:  unrecognized shape:  ARC
DEBUG:  unrecognized shape:  ARC


In [7]:
;; FIXME use global?
(define global
  (make-circuit
   #:external-pins (GND 3V3 5V VCC USB5V)))

# simple RC circuit

In [8]:
(define (RC-fixed)
         (make-circuit
              #:external-pins (In Out GND)
              #:vars ([R1 (R '1k)]
                      [C1 (C '1p)])
          #:connect (list (*- self.In R1.1)
                          (*- R1.2 C1.1 self.Out)
                          (*- C1.2 self.Out))
          #:layout (inset (vc-append R1 C1) 20)))

In [9]:
(show-layout (RC-fixed))

"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-106-15_16040017681604001768016"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-106-15_16040017681604001768029"


(pict '(picture 52.6773699298991 72.5060541200356 (put 20 20 #3=(picture 12.677369929899104 32.5060541200356 (put 0.0 26.2530270600178 #1=(prog #<procedure:...rivate/utils.rkt:1274:8> 6.253027060017801)) (put 0.0 0 #2=(prog #<procedure:...rivate/utils.rkt:1274:8> 6.253027060017801))))) 52.6773699298991 72.5060541200356 23.1265135300089 23.1265135300089 (list (child (pict #3# 12.677369929899104 32.5060541200356 3.1265135300089004 3.1265135300089004 (list (child (pict #1# 12.677369929899104 6.253027060017801 3.1265135300089004 3.1265135300089004 '() (bbox 0.0 0.0 12.677369929899104 6.253027060017801 3.1265135300089004 3.1265135300089004) #f) 0.0 26.2530270600178 1 1 0 0) (child #0=(pict #2# 12.677369929899104 6.253027060017801 3.1265135300089004 3.1265135300089004 '() (bbox 0.0 0.0 12.677369929899104 6.253027060017801 3.1265135300089004 3.1265135300089004) #f) 0.0 0 1 1 0 0)) #f #0#) 20 20 1 1 0 0)) #f #0#)

In [10]:
(circuit-plot (RC-fixed))

DEBUG:  Number of conflicts: 0
generating pdf ..
link: /tmp/out.pdf


(pict '(prog #<procedure:...rivate/utils.rkt:1274:8> 72.5060541200356) 52.6773699298991 72.5060541200356 72.5060541200356 0.0 (list (child (pict '(picture 52.6773699298991 72.5060541200356 (put 0.0 0.0 #34=(picture 52.6773699298991 72.5060541200356 (put 0 0 #30=(picture 52.6773699298991 72.5060541200356 (put 0 0 #28=(picture 52.6773699298991 72.5060541200356 (put 0.0 0.0 #16=(picture 52.6773699298991 72.5060541200356 (put 0 0 #12=(picture 52.6773699298991 72.5060541200356 (put 0 0 #8=(picture 52.6773699298991 72.5060541200356 (put 0 0 #4=(picture 52.6773699298991 72.5060541200356 (put 0 0 #2=(prog #<procedure:...rivate/utils.rkt:374:8> 72.5060541200356)) (put 20.0 46.2530270600178 #3=(prog #<procedure:...rivate/utils.rkt:1330:20> 6.253027060017801)))) (put 20.0 20.0 #7=(prog #<procedure:...rivate/utils.rkt:1330:20> 6.253027060017801)))) (put 29.711464971637234 49.3795405900267 #11=(prog #<procedure:...rivate/utils.rkt:374:8> 0)))) (put 29.711464971637234 23.1265135300089 #15=(prog #<procedure:...rivate/utils.rkt:374:8> 0)))) (put 0.0 0.0 #25=(picture 52.6773699298991 72.5060541200356 (put 0 0 #23=(picture 52.6773699298991 72.5060541200356 (put -0.0 0.0 (picture 52.6773699298991 72.5060541200356 (put 0.0 0.0 #19=(picture 52.6773699298991 72.5060541200356)) (put 0.0 0.0 (picture 52.6773699298991 72.5060541200356 (thickness long-dash (picture 52.6773699298991 72.5060541200356 (put 0 0 (picture 52.6773699298991 72.5060541200356)) (put 29.711464971637234 49.3795405900267 (line 0.0 -26.2530270600178 . #39=(#f))))))))))) (put 0.0 0.0 #24=(picture 52.6773699298991 72.5060541200356)))))) (put 22.965904958261866 23.1265135300089 #29=(prog #<procedure:...rivate/utils.rkt:374:8> 0)))) (put 29.711464971637234 23.1265135300089 #33=(prog #<procedure:...rivate/utils.rkt:374:8> 0)))) (put 0.0 0.0 #45=(picture 52.6773699298991 72.5060541200356 (put 0 0 #43=(picture 52.6773699298991 72.5060541200356 (put -0.0 0.0 (picture 52.6773699298991 72.5060541200356 (put 0.0 0.0 #38=(picture 52.6773699298991 72.5060541200356)) (put 0.0 0.0 (picture 52.6773699298991 72.5060541200356 (thickness long-dash (picture 52.6773699298991 72.5060541200356 (put 0 0 (picture 52.6773699298991 72.5060541200356)) (put 22.965904958261866 23.1265135300089 (line 6.745560013375368 0.0 . #39#)))))))))) (put 0.0 0.0 #44=(picture 52.6773699298991 72.5060541200356))))) 52.6773699298991 72.5060541200356 72.5060541200356 0.0 (list (child (pict #45# 52.6773699298991 72.5060541200356 72.5060541200356 0.0 (list (child (pict #45# 52.6773699298991 72.5060541200356 72.5060541200356 0.0 (list (child (pict #45# 52.6773699298991 72.5060541200356 72.5060541200356 0.0 (list (child #41=(pict #44# 52.6773699298991 72.5060541200356 72.5060541200356 0.0 '() #f #f) 0.0 0.0 1 1 0 0) (child #42=(pict #43# 52.6773699298991 72.5060541200356 72.5060541200356 0.0 '() (bbox 0.0 0.0 52.6773699298991 72.5060541200356 49.3795405900267 0.0) #40=(pict #38# 52.6773699298991 72.5060541200356 72.5060541200356 0.0 (list (child #37=(pict #34# 52.6773699298991 72.5060541200356 72.5060541200356 0.0 (list (child (pict #34# 52.6773699298991 72.5060541200356 72.5060541200356 0.0 (list (child (pict #33# 0 0 0 0 '() (bbox 0 0 0 0 0 0) #f) 29.711464971637234 23.1265135300089 1 1 0 0) (child #32=(pict #30# 52.6773699298991 72.5060541200356 72.5060541200356 0.0 (list (child (pict #30# 52.6773699298991 72.5060541200356 72.5060541200356 0.0 (list (child (pict #29# 0 0 0 0 '() (bbox 0 0 0 0 0 0) #f) 22.965904958261866 23.1265135300089 1 1 0 0) (child #27=(pict #28# 52.6773699298991 72.5060541200356 72.5060541200356 0.0 (list (child (pict #25# 52.6773699298991 72.5060541200356 72.5060541200356 0 (list (child (pict #25# 52.6773699298991 72.5060541200356 72.5060541200356 0 (list (child (pict #25# 52.6773699298991 72.5060541200356 72.5060541200356 0 (list (child #21=(pict #24# 52.6773699298991 72.5060541200356 72.5060541200356 0 '() (bbox 0 0 52.6773699298991 72.5060541200356 0.0 0) #f) 0.0 0.0 1 1 0 0) (child #22=(pict #23# 52.67

# A delayed push trigger

In [11]:
;; some component libraries
(define/IC (MOSFET)
           #:FP ((lcsc->fp "C727149") 1 2 3))

DEBUG:  unrecognized shape:  SVGNODE


In [12]:
(define/IC (SW_Push)
           #:FP ((kicad->fp "Button_Switch_THT.pretty/"
                                 "SW_PUSH_6mm.kicad_mod")
                 1 1 2 2)
           #:LEFT 1
           #:RIGHT 2)

In [13]:
(define (delay-push-fixed [btn #f])
  (make-circuit
      #:external-pins (VCC GND Ctrl)
      #:vars ((RC1 (RC-fixed))
              [D1 (Diode '1N4148)]
              [R4 (R '1k)]
              [SW1 (if btn btn (SW_Push))]
              [Q1 (MOSFET '2N7002)]
              [R2 (R '10k)]
              [D2 (LED)])
      #:connect (list (*- RC1.Out D1.anode Q1.1)
                      (*- RC1.GND Q1.2)
                      (*- Q1.3 self.Ctrl))
   #:connect (list (*- self.VCC SW1 D1.cathode RC1.In self.GND)
                   (*- self.VCC D2 self.Ctrl))
   ;; TODO implement a default layout
;;    #:layout (pict:rectangle 200 200)
   #:layout (inset SW1 50)
   ))

In [14]:
;; FIXME this does not seem to be deterministic
;; TODO implement copy-on-assign semantic, like pict?
(parameterize ([current-directory "./out/onebutton/delay-push-fixed"]
               [padding-general 2])
    (circuit-export (delay-push-fixed)
                    #:auto-place #t
                    #:formats '(kicad pdf png)))

"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-106-15_16040017681604001768345"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-106-15_16040017681604001768357"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-106-15_16040017681604001768367"
DEBUG:  sending for placement ..
DEBUG:  saving ..
generating KiCAD PCB ..
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-137-15_16040017691604001769296"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-137-15_16040017691604001769307"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-137-15_16040017691604001769313"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-137-15_16040017691604001769324"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-137-15_16040017691604001769340"
link: /home/jovyan/bhdl/bhdl-test/out/onebutton/delay-push-fixed/out.kicad_pcb
DEBUG:  Number of conflicts: 0
generating pdf ..
link: /home/jovyan/bhdl/bhdl-test/out/onebutton/delay-push-fixed/out.

(pict '(prog #<procedure:...rivate/utils.rkt:1274:8> 132.12171434940652) 140.68750484258157 132.12171434940652 132.12171434940652 0.0 (list (child (pict '(picture 140.68750484258157 132.12171434940652 (put 0.0 0.0 #163=(picture 140.68750484258157 132.12171434940652 (put 0 0 #159=(picture 140.68750484258157 132.12171434940652 (put 0 0 #157=(picture 140.68750484258157 132.12171434940652 (put 0.0 0.0 #145=(picture 140.68750484258157 132.12171434940652 (put 0 0 #141=(picture 140.68750484258157 132.12171434940652 (put 0 0 #139=(picture 140.68750484258157 132.12171434940652 (put 0.0 0.0 #127=(picture 140.68750484258157 132.12171434940652 (put 0 0 #123=(picture 140.68750484258157 132.12171434940652 (put 0 0 #121=(picture 140.68750484258157 132.12171434940652 (put 0.0 0.0 #109=(picture 140.68750484258157 132.12171434940652 (put 0 0 #105=(picture 140.68750484258157 132.12171434940652 (put 0 0 #103=(picture 140.68750484258157 132.12171434940652 (put 0.0 0.0 #91=(picture 140.68750484258157 132.12171434940652 (put 0 0 #87=(picture 140.68750484258157 132.12171434940652 (put 0 0 #85=(picture 140.68750484258157 132.12171434940652 (put 0.0 0.0 #73=(picture 140.68750484258157 132.12171434940652 (put 0 0 #69=(picture 140.68750484258157 132.12171434940652 (put 0 0 #67=(picture 140.68750484258157 132.12171434940652 (put 0.0 0.0 #54=(picture 140.68750484258157 132.12171434940652 (put 0 0 #50=(picture 140.68750484258157 132.12171434940652 (put 0 0 #48=(picture 140.68750484258157 132.12171434940652 (put 0.0 0.0 #36=(picture 140.68750484258157 132.12171434940652 (put 0 0 #32=(picture 140.68750484258157 132.12171434940652 (put 0 0 #28=(picture 140.68750484258157 132.12171434940652 (put 0 0 #23=(picture 140.68750484258157 132.12171434940652 (put 0 0 #18=(picture 140.68750484258157 132.12171434940652 (put 0 0 #13=(picture 140.68750484258157 132.12171434940652 (put 0 0 #8=(picture 140.68750484258157 132.12171434940652 (put 0 0 #4=(picture 140.68750484258157 132.12171434940652 (put 0 0 #2=(prog #<procedure:...rivate/utils.rkt:374:8> 132.12171434940652)) (put 64.09094903505044 34.34981081939762 #3=(prog #<procedure:...rivate/utils.rkt:1330:20> 6.253027060017801)))) (put 106.93675503505045 101.29382281939762 #7=(prog #<procedure:...rivate/utils.rkt:1330:20> 6.253027060017801)))) (put 56.07183910158457 90.7568912911721 #12=(prog #<procedure:...rivate/utils.rkt:1330:20> 10.707238116468838)))) (put 49.99999757870921 49.99999717470325 #17=(prog #<procedure:...rivate/utils.rkt:1330:20> 32.12171434940652)))) (put 4.282895129965525 62.79034680907526 #22=(prog #<procedure:...rivate/utils.rkt:1330:20> 13.490821080662524)))) (put 4.28289610158458 99.3189912911721 #27=(prog #<procedure:...rivate/utils.rkt:1330:20> 10.707238116468838)))) (put 84.2631595514095 75.69736865422847 #31=(prog #<procedure:...rivate/utils.rkt:374:8> 0)))) (put 8.779936110501492 104.67261034940651 #35=(prog #<procedure:...rivate/utils.rkt:374:8> 0)))) (put 0.0 0.0 #45=(picture 140.68750484258157 132.12171434940652 (put 0 0 #43=(picture 140.68750484258157 132.12171434940652 (put -0.0 0.0 (picture 140.68750484258157 132.12171434940652 (put 0.0 0.0 #39=(picture 140.68750484258157 132.12171434940652)) (put 0.0 0.0 (picture 140.68750484258157 132.12171434940652 (thickness long-dash (picture 140.68750484258157 132.12171434940652 (put 0 0 (picture 140.68750484258157 132.12171434940652)) (put 84.2631595514095 75.69736865422847 (line -75.483223440908 28.975241695178042 . #58=(#f))))))))))) (put 0.0 0.0 #44=(picture 140.68750484258157 132.12171434940652)))))) (put 5.781905468244559 69.53575734940651 #49=(prog #<procedure:...rivate/utils.rkt:374:8> 0)))) (put 60.568879110501484 96.11051034940652 #53=(prog #<procedure:...rivate/utils.rkt:374:8> 0)))) (put 0.0 0.0 #64=(picture 140.68750484258157 132.12171434940652 (put 0 0 #62=(picture 140.68750484258157 132.12171434940652 (put -0.0 0.0 (picture 140.68750484258157 132.12171434940652 (put 0.0 0.0 #57=(picture 140.68750484258157 132.12171434940652)) (put 0.0 

# connect ESP-01 module

In [15]:
(define/IC (ESP01)
           #:FP ((lcsc->fp "C503581") 
                 GND GPIO2 GPIO0 URXD
                 UTXD CH_PD RST VCC))

DEBUG:  unrecognized shape:  TEXT
DEBUG:  unrecognized shape:  CIRCLE


In [16]:
(define/IC (OneBattery)
           ;; FIXME using pinheader for battery holder?
           #:FP ((kicad->fp "kicad-footprints" 
                           "Connector_PinHeader_2.54mm.pretty"
                           "PinHeader_1x02_P2.54mm_Horizontal.kicad_mod")
                 pos neg))

In [17]:
(atom->fp-pict (OneBattery))

"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-106-15_16040017691604001769422"


(pict '(prog #<procedure:...rivate/utils.rkt:1274:8> 26.33980576651334) 52.89375629535606 26.33980576651334 7.709211443857564 18.630594322655778 (list (child (pict '(picture 12.350000000000001 6.1499999999999995 (put 1.8 4.35 #0=(prog #<procedure:...erber-viewer.rkt:162:18> 0))) 12.350000000000001 6.1499999999999995 1.8 4.35 (list (child (pict #0# 0 0 0 0 '() (bbox 0 0 0 0 0 0) #f) 1.8 4.35 1 1 0 0)) (bbox 0.0 0.0 12.350000000000001 6.1499999999999995 4.35 4.35) #f) 0 0 4.282895246587535 4.282895246587535 0 0)) (bbox 0.0 0.0 52.89375629535606 26.33980576651334 18.630594322655778 18.630594322655778) #f)

In [18]:
(define one-button
  (make-circuit 
   #:vars ([btn (SW_Push)]
           [delay_sw1 (delay-push-fixed btn)]
           [R5 (R '10k)]
           [R6 (R '10k)]
           [BT1 (OneBattery '3V)]
           [U1 (ESP01)])
   #:connect (list 
              ;; FIXME this is not really parallel connection, but split connection
;;               (*= [BT1.pos -  U1.VCC]
;;                        [BT1.pos R5 U1.RST]
;;                        [BT1.pos R6 U1.CH_PD]
;;                        [BT1.pos -  delay_sw1.VCC])
              (*- BT1.pos (*< (*- R5 U1.RST)
                              (*- R6 U1.CH_PD)
                              U1.VCC
                              delay_sw1.VCC))
                   (*- BT1.neg U1.GND delay_sw1.GND)
                   (*- U1.RST delay_sw1.Ctrl))
   ;; TODO layout
;;    #:layout (pict:rectangle 300 300)
   #:layout (inset (hc-append 200 btn U1)
                   0 20 0 20)))

"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-106-15_16040017691604001769459"


In [19]:
(show-layout one-button)

(pict '(picture 346.47502171618896 101.76788526601871 (put 0 20 #3=(picture 346.47502171618896 61.76788526601871 (put 0 14.823085458306096 #1=(prog #<procedure:...rivate/utils.rkt:1274:8> 32.12171434940652)) (put 240.68750484258157 0.0 #2=(prog #<procedure:...rivate/utils.rkt:1274:8> 61.76788526601871))))) 346.47502171618896 101.76788526601871 41.2474283281874 50.45995099373055 (list (child (pict #3# 346.47502171618896 61.76788526601871 21.247428328187397 30.45995099373055 (list (child (pict #1# 40.68750484258158 32.12171434940652 6.424342869881302 25.69737147952521 '() (bbox 0 0 40.68750484258158 32.12171434940652 25.697371479525216 25.69737147952521) #f) 0 14.823085458306096 1 1 0 0) (child #0=(pict #2# 105.78751687360737 61.76788526601871 31.307934272288158 30.45995099373055 '() (bbox 0.0 0.0 105.78751687360737 61.76788526601871 30.45995099373055 30.45995099373055) #f) 240.68750484258157 0.0 1 1 0 0)) #f #0#) 0 20 1 1 0 0)) #f #0#)

In [20]:
;; FIXME this does not seem to be deterministic
;; TODO implement copy-on-assign semantic, like pict?
(parameterize ([current-directory "./out/onebutton/one"]
               [padding-general 1])
    (circuit-export one-button
                    #:auto-place #t
                    #:formats '(kicad pdf png)))

DEBUG:  sending for placement ..
DEBUG:  saving ..
generating KiCAD PCB ..
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-137-15_16040017701604001770970"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-137-15_16040017701604001770990"
link: /home/jovyan/bhdl/bhdl-test/out/onebutton/one/out.kicad_pcb
DEBUG:  Number of conflicts: 0
generating pdf ..
link: /home/jovyan/bhdl/bhdl-test/out/onebutton/one/out.pdf
link: /home/jovyan/bhdl/bhdl-test/out/onebutton/one/out.png


(pict '(prog #<procedure:...rivate/utils.rkt:1274:8> 101.76788526601871) 346.47502171618896 101.76788526601871 101.76788526601871 0.0 (list (child (pict '(picture 346.47502171618896 101.76788526601871 (put 0.0 0.0 #380=(picture 346.47502171618896 101.76788526601871 (put 0 0 #376=(picture 346.47502171618896 101.76788526601871 (put 0 0 #374=(picture 346.47502171618896 101.76788526601871 (put 0.0 0.0 #362=(picture 346.47502171618896 101.76788526601871 (put 0 0 #358=(picture 346.47502171618896 101.76788526601871 (put 0 0 #356=(picture 346.47502171618896 101.76788526601871 (put 0.0 0.0 #344=(picture 346.47502171618896 101.76788526601871 (put 0 0 #340=(picture 346.47502171618896 101.76788526601871 (put 0 0 #338=(picture 346.47502171618896 101.76788526601871 (put 0.0 0.0 #326=(picture 346.47502171618896 101.76788526601871 (put 0 0 #322=(picture 346.47502171618896 101.76788526601871 (put 0 0 #320=(picture 346.47502171618896 101.76788526601871 (put 0.0 0.0 #308=(picture 346.47502171618896 101.76788526601871 (put 0 0 #304=(picture 346.47502171618896 101.76788526601871 (put 0 0 #302=(picture 346.47502171618896 101.76788526601871 (put 0.0 0.0 #290=(picture 346.47502171618896 101.76788526601871 (put 0 0 #286=(picture 346.47502171618896 101.76788526601871 (put 0 0 #284=(picture 346.47502171618896 101.76788526601871 (put 0.0 0.0 #272=(picture 346.47502171618896 101.76788526601871 (put 0 0 #268=(picture 346.47502171618896 101.76788526601871 (put 0 0 #266=(picture 346.47502171618896 101.76788526601871 (put 0.0 0.0 #254=(picture 346.47502171618896 101.76788526601871 (put 0 0 #250=(picture 346.47502171618896 101.76788526601871 (put 0 0 #248=(picture 346.47502171618896 101.76788526601871 (put 0.0 0.0 #236=(picture 346.47502171618896 101.76788526601871 (put 0 0 #232=(picture 346.47502171618896 101.76788526601871 (put 0 0 #230=(picture 346.47502171618896 101.76788526601871 (put 0.0 0.0 #218=(picture 346.47502171618896 101.76788526601871 (put 0 0 #214=(picture 346.47502171618896 101.76788526601871 (put 0 0 #212=(picture 346.47502171618896 101.76788526601871 (put 0.0 0.0 #200=(picture 346.47502171618896 101.76788526601871 (put 0 0 #196=(picture 346.47502171618896 101.76788526601871 (put 0 0 #194=(picture 346.47502171618896 101.76788526601871 (put 0.0 0.0 #182=(picture 346.47502171618896 101.76788526601871 (put 0 0 #178=(picture 346.47502171618896 101.76788526601871 (put 0 0 #176=(picture 346.47502171618896 101.76788526601871 (put 0.0 0.0 #164=(picture 346.47502171618896 101.76788526601871 (put 0 0 #160=(picture 346.47502171618896 101.76788526601871 (put 0 0 #158=(picture 346.47502171618896 101.76788526601871 (put 0.0 0.0 #146=(picture 346.47502171618896 101.76788526601871 (put 0 0 #142=(picture 346.47502171618896 101.76788526601871 (put 0 0 #140=(picture 346.47502171618896 101.76788526601871 (put 0.0 0.0 #128=(picture 346.47502171618896 101.76788526601871 (put 0 0 #124=(picture 346.47502171618896 101.76788526601871 (put 0 0 #122=(picture 346.47502171618896 101.76788526601871 (put 0.0 0.0 #110=(picture 346.47502171618896 101.76788526601871 (put 0 0 #106=(picture 346.47502171618896 101.76788526601871 (put 0 0 #104=(picture 346.47502171618896 101.76788526601871 (put 0.0 0.0 #92=(picture 346.47502171618896 101.76788526601871 (put 0 0 #88=(picture 346.47502171618896 101.76788526601871 (put 0 0 #86=(picture 346.47502171618896 101.76788526601871 (put 0.0 0.0 #73=(picture 346.47502171618896 101.76788526601871 (put 0 0 #69=(picture 346.47502171618896 101.76788526601871 (put 0 0 #67=(picture 346.47502171618896 101.76788526601871 (put 0.0 0.0 #55=(picture 346.47502171618896 101.76788526601871 (put 0 0 #51=(picture 346.47502171618896 101.76788526601871 (put 0 0 #47=(picture 346.47502171618896 101.76788526601871 (put 0 0 #42=(picture 346.47502171618896 101.76788526601871 (put 0 0 #37=(picture 346.47502171618896 101.76788526601871 (put 0 0 #33=(picture 346.47502171618896 101.76788526601871 (put 0 0 #28=(picture 346.47502171618896 101.76788526601871 (put 0 0 #23=(p

# More delay pushbuttons

In [21]:
(define three-button
  (make-circuit 
   #:vars ([buttons (for/list ([row (in-range 3)])
                              (SW_Push))]
           [button-row (for/list ([btn buttons])
                                 (delay-push-fixed btn))]
;;            [button-row (for/list ([row (in-range 3)])
;;                                 (delay-push-fixed))]
           ;; TODO use thr-hole footprint for resistors
           [R5 (R '10k)]
           [R6 (R '10k)]
           [BT1 (OneBattery '3V)]
           [U1 (ESP01)])
   #:connect (list 
              (*- BT1.pos (*< (*- R5 U1.RST)
                              (*- R6 U1.CH_PD)
                              U1.VCC))
              ;; FIXME
              (for/list ([row (in-range 3)])
                        (*- BT1.pos (pin-ref (list-ref button-row row) 'VCC)))
              ;; neg to GNDs
               (for/list ([btn button-row])
                         (*- BT1.neg U1.GND btn.GND))
              ;; connect GPIOs
              (for/list ([btn button-row])
                        (*- U1.RST btn.Ctrl))
               )
   #:layout (inset (hc-append 300 (vc-append buttons ..) U1)
                   0)))

In [22]:
(pict:frame (show-layout three-button))

(pict '(picture 446.47502171618896 136.36514304821955 (put -0.0 0.0 #17=(picture 446.47502171618896 136.36514304821955 (put 0.0 0.0 #15=(picture 446.47502171618896 136.36514304821955 (put 0 0 #8=(picture 446.47502171618896 136.36514304821955 (put 0 0.0 #6=(picture 40.68750484258158 136.36514304821955 (put 0.0 104.24342869881303 #2=(prog #<procedure:...rivate/utils.rkt:1274:8> 32.12171434940652)) (put 0.0 0 #5=(picture 40.68750484258158 84.24342869881303 (put 0.0 52.12171434940652 #2#) (put 0.0 0 #2#))))) (put 340.6875048425816 37.29862889110042 #7=(prog #<procedure:...rivate/utils.rkt:1274:8> 61.76788526601871)))))) (put 0.0 0.0 #16=(picture 446.47502171618896 136.36514304821955 (put 0 0 #14=(picture 446.47502171618896 136.36514304821955 (put 0 0 #12=(picture 446.47502171618896 136.36514304821955)) (put 340.6875048425816 37.29862889110042 #13=(picture 105.78751687360737 61.76788526601871)))) (put 0 0 (picture 446.47502171618896 0 (put 0 0 (line 1 0 446.47502171618896)))) (put 0 136.36514304821955 (picture 446.47502171618896 0 (put 0 0 (line 1 0 446.47502171618896)))) (put 0 0 (picture 0 136.36514304821955 (put 0 0 (line 0 1 136.36514304821955)))) (put 446.47502171618896 0 (picture 0 136.36514304821955 (put 0 0 (line 0 1 136.36514304821955))))))))) 446.47502171618896 136.36514304821955 6.424342869881302 25.69737147952521 (list (child (pict #17# 446.47502171618896 136.36514304821955 6.424342869881302 25.69737147952521 (list (child (pict #16# 446.47502171618896 136.36514304821955 6.424342869881302 25.69737147952521 (list (child (pict #14# 446.47502171618896 136.36514304821955 6.424342869881302 25.69737147952521 (list (child (pict #14# 446.47502171618896 136.36514304821955 6.424342869881302 25.69737147952521 (list (child (pict #14# 446.47502171618896 136.36514304821955 6.424342869881302 25.69737147952521 (list (child #10=(pict #13# 105.78751687360737 61.76788526601871 31.307934272288158 30.45995099373055 '() #f #f) 340.6875048425816 37.29862889110042 1 1 0 0) (child #11=(pict #12# 446.47502171618896 136.36514304821955 6.424342869881302 25.69737147952521 '() (bbox 0.0 0.0 446.47502171618896 136.36514304821955 129.94080017833824 25.69737147952521) #0=(pict #7# 105.78751687360737 61.76788526601871 31.307934272288158 30.45995099373055 '() (bbox 0.0 0.0 105.78751687360737 61.76788526601871 30.45995099373055 30.45995099373055) #f)) 0 0 1 1 0 0)) #f #f) 0 0 1 1 0 0)) #f (list #11# #0#)) 0 0 1 1 0 0)) #f #10#) 0 0 1 1 0 0)) #f #10#) 0.0 0.0 1 1 0 0) (child #9=(pict #15# 446.47502171618896 136.36514304821955 6.424342869881302 25.69737147952521 (list (child (pict #8# 446.47502171618896 136.36514304821955 6.424342869881302 25.69737147952521 (list (child (pict #6# 40.68750484258158 136.36514304821955 6.424342869881302 25.69737147952521 (list (child (pict #2# 40.68750484258158 32.12171434940652 6.424342869881302 25.69737147952521 '() #1=(bbox 0 0 40.68750484258158 32.12171434940652 25.697371479525216 25.69737147952521) #f) 0.0 104.24342869881303 1 1 0 0) (child #4=(pict #5# 40.68750484258158 84.24342869881303 6.424342869881302 25.69737147952521 (list (child (pict #2# 40.68750484258158 32.12171434940652 6.424342869881302 25.69737147952521 '() #1# #f) 0.0 52.12171434940652 1 1 0 0) (child #3=(pict #2# 40.68750484258158 32.12171434940652 6.424342869881302 25.69737147952521 '() #1# #f) 0.0 0 1 1 0 0)) (bbox 0.0 0.0 40.68750484258158 84.24342869881303 77.81908582893173 25.69737147952521) #3#) 0.0 0 1 1 0 0)) (bbox 0.0 0.0 40.68750484258158 136.36514304821955 129.94080017833824 25.69737147952521) (list #4# #3#)) 0 0.0 1 1 0 0) (child #0# 340.6875048425816 37.29862889110042 1 1 0 0)) (bbox 0.0 0.0 446.47502171618896 136.36514304821955 129.94080017833824 25.69737147952521) #0#) 0 0 1 1 0 0)) (bbox 0.0 0.0 446.47502171618896 136.36514304821955 129.94080017833824 25.69737147952521) #0#) 0.0 0.0 1 1 0 0)) #f (list #9# #0#)) -0.0 0.0 1 1 0 0)) #f (list #9# #0#))

In [23]:
;; FIXME this does not seem to be deterministic
;; TODO implement copy-on-assign semantic, like pict?
(parameterize ([current-directory "./out/onebutton/three"]
               [padding-general 1])
    (circuit-export three-button
                    #:auto-place #t
                    #:formats '(kicad pdf png)))

DEBUG:  sending for placement ..
DEBUG:  saving ..
generating KiCAD PCB ..
link: /home/jovyan/bhdl/bhdl-test/out/onebutton/three/out.kicad_pcb
DEBUG:  Number of conflicts: 0
generating pdf ..
link: /home/jovyan/bhdl/bhdl-test/out/onebutton/three/out.pdf
link: /home/jovyan/bhdl/bhdl-test/out/onebutton/three/out.png


(pict '(prog #<procedure:...rivate/utils.rkt:1274:8> 136.36514304821955) 446.47502171618896 136.36514304821955 136.36514304821955 0.0 (list (child (pict '(picture 446.47502171618896 136.36514304821955 (put 0.0 0.0 #827=(picture 446.47502171618896 136.36514304821955 (put 0 0 #823=(picture 446.47502171618896 136.36514304821955 (put 0 0 #821=(picture 446.47502171618896 136.36514304821955 (put 0.0 0.0 #809=(picture 446.47502171618896 136.36514304821955 (put 0 0 #805=(picture 446.47502171618896 136.36514304821955 (put 0 0 #803=(picture 446.47502171618896 136.36514304821955 (put 0.0 0.0 #791=(picture 446.47502171618896 136.36514304821955 (put 0 0 #787=(picture 446.47502171618896 136.36514304821955 (put 0 0 #785=(picture 446.47502171618896 136.36514304821955 (put 0.0 0.0 #773=(picture 446.47502171618896 136.36514304821955 (put 0 0 #769=(picture 446.47502171618896 136.36514304821955 (put 0 0 #767=(picture 446.47502171618896 136.36514304821955 (put 0.0 0.0 #755=(picture 446.47502171618896 136.36514304821955 (put 0 0 #751=(picture 446.47502171618896 136.36514304821955 (put 0 0 #749=(picture 446.47502171618896 136.36514304821955 (put 0.0 0.0 #737=(picture 446.47502171618896 136.36514304821955 (put 0 0 #733=(picture 446.47502171618896 136.36514304821955 (put 0 0 #731=(picture 446.47502171618896 136.36514304821955 (put 0.0 0.0 #719=(picture 446.47502171618896 136.36514304821955 (put 0 0 #715=(picture 446.47502171618896 136.36514304821955 (put 0 0 #713=(picture 446.47502171618896 136.36514304821955 (put 0.0 0.0 #701=(picture 446.47502171618896 136.36514304821955 (put 0 0 #697=(picture 446.47502171618896 136.36514304821955 (put 0 0 #695=(picture 446.47502171618896 136.36514304821955 (put 0.0 0.0 #683=(picture 446.47502171618896 136.36514304821955 (put 0 0 #679=(picture 446.47502171618896 136.36514304821955 (put 0 0 #677=(picture 446.47502171618896 136.36514304821955 (put 0.0 0.0 #665=(picture 446.47502171618896 136.36514304821955 (put 0 0 #661=(picture 446.47502171618896 136.36514304821955 (put 0 0 #659=(picture 446.47502171618896 136.36514304821955 (put 0.0 0.0 #647=(picture 446.47502171618896 136.36514304821955 (put 0 0 #643=(picture 446.47502171618896 136.36514304821955 (put 0 0 #641=(picture 446.47502171618896 136.36514304821955 (put 0.0 0.0 #629=(picture 446.47502171618896 136.36514304821955 (put 0 0 #625=(picture 446.47502171618896 136.36514304821955 (put 0 0 #623=(picture 446.47502171618896 136.36514304821955 (put 0.0 0.0 #611=(picture 446.47502171618896 136.36514304821955 (put 0 0 #607=(picture 446.47502171618896 136.36514304821955 (put 0 0 #605=(picture 446.47502171618896 136.36514304821955 (put 0.0 0.0 #593=(picture 446.47502171618896 136.36514304821955 (put 0 0 #589=(picture 446.47502171618896 136.36514304821955 (put 0 0 #587=(picture 446.47502171618896 136.36514304821955 (put 0.0 0.0 #575=(picture 446.47502171618896 136.36514304821955 (put 0 0 #571=(picture 446.47502171618896 136.36514304821955 (put 0 0 #569=(picture 446.47502171618896 136.36514304821955 (put 0.0 0.0 #557=(picture 446.47502171618896 136.36514304821955 (put 0 0 #553=(picture 446.47502171618896 136.36514304821955 (put 0 0 #551=(picture 446.47502171618896 136.36514304821955 (put 0.0 0.0 #539=(picture 446.47502171618896 136.36514304821955 (put 0 0 #535=(picture 446.47502171618896 136.36514304821955 (put 0 0 #533=(picture 446.47502171618896 136.36514304821955 (put 0.0 0.0 #521=(picture 446.47502171618896 136.36514304821955 (put 0 0 #517=(picture 446.47502171618896 136.36514304821955 (put 0 0 #515=(picture 446.47502171618896 136.36514304821955 (put 0.0 0.0 #503=(picture 446.47502171618896 136.36514304821955 (put 0 0 #499=(picture 446.47502171618896 136.36514304821955 (put 0 0 #497=(picture 446.47502171618896 136.36514304821955 (put 0.0 0.0 #485=(picture 446.47502171618896 136.36514304821955 (put 0 0 #481=(picture 446.47502171618896 136.36514304821955 (put 0 0 #479=(picture 446.47502171618896 136.36514304821955 (put 0.0 0.0 #467=(picture 446.47502171618896 136.365143048

# Calculating RC values

In [24]:
(define (get-C tau R)
  (/ tau R))

In [25]:
(define (RC-flexible tau)
  (make-circuit
   #:external-pins (In Out GND)
   #:vars ([R1 (R 1000)]
           [C1 (C (get-C tau 1000))])
   #:connect (list (*- self.In R1.1)
                   (*- R1.2 C1.1 self.Out)
                   (*- C1.2 self.Out))
   #:layout (inset (vc-append R1 C1) 20)))

In [26]:
(RC-flexible 1)

#<Composite-265597>

In [27]:
;; FIXME this does not seem to be deterministic
;; TODO implement copy-on-assign semantic, like pict?
(parameterize ([current-directory "./out/onebutton/flex"]
               [padding-general 2])
    (circuit-export (RC-flexible 1)
                    #:auto-place #f
                    #:formats '(kicad pdf png bom)))

generating KiCAD PCB ..
link: /home/jovyan/bhdl/bhdl-test/out/onebutton/flex/out.kicad_pcb
Generating BOM ..
DEBUG:  Number of conflicts: 0
generating pdf ..
link: /home/jovyan/bhdl/bhdl-test/out/onebutton/flex/out.pdf
link: /home/jovyan/bhdl/bhdl-test/out/onebutton/flex/out.png


(pict '(prog #<procedure:...rivate/utils.rkt:1274:8> 72.5060541200356) 52.6773699298991 72.5060541200356 72.5060541200356 0.0 (list (child (pict '(picture 52.6773699298991 72.5060541200356 (put 0.0 0.0 #34=(picture 52.6773699298991 72.5060541200356 (put 0 0 #30=(picture 52.6773699298991 72.5060541200356 (put 0 0 #28=(picture 52.6773699298991 72.5060541200356 (put 0.0 0.0 #16=(picture 52.6773699298991 72.5060541200356 (put 0 0 #12=(picture 52.6773699298991 72.5060541200356 (put 0 0 #8=(picture 52.6773699298991 72.5060541200356 (put 0 0 #4=(picture 52.6773699298991 72.5060541200356 (put 0 0 #2=(prog #<procedure:...rivate/utils.rkt:374:8> 72.5060541200356)) (put 20.0 46.2530270600178 #3=(prog #<procedure:...rivate/utils.rkt:1330:20> 6.253027060017801)))) (put 20.0 20.0 #7=(prog #<procedure:...rivate/utils.rkt:1330:20> 6.253027060017801)))) (put 29.711464971637234 49.3795405900267 #11=(prog #<procedure:...rivate/utils.rkt:374:8> 0)))) (put 29.711464971637234 23.1265135300089 #15=(prog #<procedure:...rivate/utils.rkt:374:8> 0)))) (put 0.0 0.0 #25=(picture 52.6773699298991 72.5060541200356 (put 0 0 #23=(picture 52.6773699298991 72.5060541200356 (put -0.0 0.0 (picture 52.6773699298991 72.5060541200356 (put 0.0 0.0 #19=(picture 52.6773699298991 72.5060541200356)) (put 0.0 0.0 (picture 52.6773699298991 72.5060541200356 (thickness long-dash (picture 52.6773699298991 72.5060541200356 (put 0 0 (picture 52.6773699298991 72.5060541200356)) (put 29.711464971637234 49.3795405900267 (line 0.0 -26.2530270600178 . #39=(#f))))))))))) (put 0.0 0.0 #24=(picture 52.6773699298991 72.5060541200356)))))) (put 22.965904958261866 23.1265135300089 #29=(prog #<procedure:...rivate/utils.rkt:374:8> 0)))) (put 29.711464971637234 23.1265135300089 #33=(prog #<procedure:...rivate/utils.rkt:374:8> 0)))) (put 0.0 0.0 #45=(picture 52.6773699298991 72.5060541200356 (put 0 0 #43=(picture 52.6773699298991 72.5060541200356 (put -0.0 0.0 (picture 52.6773699298991 72.5060541200356 (put 0.0 0.0 #38=(picture 52.6773699298991 72.5060541200356)) (put 0.0 0.0 (picture 52.6773699298991 72.5060541200356 (thickness long-dash (picture 52.6773699298991 72.5060541200356 (put 0 0 (picture 52.6773699298991 72.5060541200356)) (put 22.965904958261866 23.1265135300089 (line 6.745560013375368 0.0 . #39#)))))))))) (put 0.0 0.0 #44=(picture 52.6773699298991 72.5060541200356))))) 52.6773699298991 72.5060541200356 72.5060541200356 0.0 (list (child (pict #45# 52.6773699298991 72.5060541200356 72.5060541200356 0.0 (list (child (pict #45# 52.6773699298991 72.5060541200356 72.5060541200356 0.0 (list (child (pict #45# 52.6773699298991 72.5060541200356 72.5060541200356 0.0 (list (child #41=(pict #44# 52.6773699298991 72.5060541200356 72.5060541200356 0.0 '() #f #f) 0.0 0.0 1 1 0 0) (child #42=(pict #43# 52.6773699298991 72.5060541200356 72.5060541200356 0.0 '() (bbox 0.0 0.0 52.6773699298991 72.5060541200356 49.3795405900267 0.0) #40=(pict #38# 52.6773699298991 72.5060541200356 72.5060541200356 0.0 (list (child #37=(pict #34# 52.6773699298991 72.5060541200356 72.5060541200356 0.0 (list (child (pict #34# 52.6773699298991 72.5060541200356 72.5060541200356 0.0 (list (child (pict #33# 0 0 0 0 '() (bbox 0 0 0 0 0 0) #f) 29.711464971637234 23.1265135300089 1 1 0 0) (child #32=(pict #30# 52.6773699298991 72.5060541200356 72.5060541200356 0.0 (list (child (pict #30# 52.6773699298991 72.5060541200356 72.5060541200356 0.0 (list (child (pict #29# 0 0 0 0 '() (bbox 0 0 0 0 0 0) #f) 22.965904958261866 23.1265135300089 1 1 0 0) (child #27=(pict #28# 52.6773699298991 72.5060541200356 72.5060541200356 0.0 (list (child (pict #25# 52.6773699298991 72.5060541200356 72.5060541200356 0 (list (child (pict #25# 52.6773699298991 72.5060541200356 72.5060541200356 0 (list (child (pict #25# 52.6773699298991 72.5060541200356 72.5060541200356 0 (list (child #21=(pict #24# 52.6773699298991 72.5060541200356 72.5060541200356 0 '() (bbox 0 0 52.6773699298991 72.5060541200356 0.0 0) #f) 0.0 0.0 1 1 0 0) (child #22=(pict #23# 52.67

In [28]:
;; read the CSV file
(call-with-input-file "out/onebutton/flex/BOM.csv"
                      (lambda (in)
                        (displayln (port->string in))))

Annotation, Name, Footprint, Values
R1,Resistor/R,0603,(1000)
C2,Capacitor/C,#f,(1/1000)
